# Comentarios sobre linearWithPandas


In [1]:
!pip install -q sklearn

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

## Load the titanic dataset
You will use the Titanic dataset with the (rather morbid) goal of predicting passenger survival, given characteristics such as gender, age, class, etc.

In [3]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [4]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

## Explore the data

The dataset contains the following features

In [5]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [6]:
dftrain.shape[0], dfeval.shape[0]

(627, 264)

In [7]:
dftrain.shape,dfeval.shape

((627, 9), (264, 9))

## Feature Engineering for the Model

### Base Feature Columns

Con Pandas tenemos la opción de poder convertir una columna categorica a one-shot. Lo podemos ver en acción con la Serie sex. Más adelante vemos el uso más estandar con `feature_columns`:

In [8]:
dftrain['sex'] = pd.Categorical(dftrain['sex'])
dftrain['sex'] = dftrain.sex.cat.codes
dfeval['sex'] = pd.Categorical(dfeval['sex'])
dfeval['sex'] = dfeval.sex.cat.codes

In [9]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,1,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,0,38.0,1,0,71.2833,First,C,Cherbourg,n
2,0,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,0,35.0,1,0,53.1000,First,C,Southampton,n
4,1,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [10]:
CATEGORICAL_COLUMNS = ['n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare','sex']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

En `feature_columns` lo que tenemos es un diccionario con el criterio que vamos a utilizar para convertir el dataset en un conjunto de datasets categoricos o numericos.

Los categoricos se definen con `tf.feature_column.categorical_column_with_vocabulary_list(nombre, vocabulary)`

- El primer argumento es el nombre de la feature
- El segundo argumento es el diccionario, el dominio de valores a categorizar

Los numéricos se definen con `tf.feature_column.numeric_column(feature_name, dtype=tf.float32)`

- El primer argumento es el nombre de la feature
- El segundo argumento es el tipo de dato


In [11]:
feature_columns

[VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None

In [12]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    #Creamos un diccionario a partir del dataframe Panda. Esto es la clave
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

Aqui vamos a definir una funcion que devuelve una funcion. La funcion que devolvemos es una funcion queno tiene argumentos y que nos devolvera un dataset cada vez que la llamemos `input_function()`.

El dataset que nos devuelve tiene la siguiente particularidad:

- Podemos controlar el tamaño de la muestra con el argumento `batch_size`
- Podemos controlar si los datos se van a barajar. Si se barajan se barajan en grupos de 1000
- Podemos indicar cuantas veces vamos a consumir los datos - cuantos epochs

Antes de utilizar la funcion para crear el dataset para el entrenamiento, echemos un vistazo al dataset, para entender como manejarlo.

### Analizar el Dataset

El factor de forma es:

In [13]:
dftrain.shape, y_train.shape

((627, 9), (627,))

Tenemos 627 muestras. Cada muestra consiste de 9 features. Los tipos de cada feature, y de la salida son:

In [14]:
dftrain.dtypes,y_train.dtypes

(sex                      int8
 age                   float64
 n_siblings_spouses      int64
 parch                   int64
 fare                  float64
 class                  object
 deck                   object
 embark_town            object
 alone                  object
 dtype: object,
 dtype('int64'))

Veamos que pinta tienen los datos:

In [15]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,1,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,0,38.0,1,0,71.2833,First,C,Cherbourg,n
2,0,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,0,35.0,1,0,53.1000,First,C,Southampton,n
4,1,28.0,0,0,8.4583,Third,unknown,Queenstown,y


Para utilizar los dataframes en el entrenamiento podemos covertirlo en un diccionario. Al hacerlo cada serie - columna - del datastream se convierte en una key del diccionario. El tipo de datos es el que tenía en el datastream. Por ejemplo, la serie `class` quedaría:

In [16]:
dict(dftrain).get('class').shape,dict(dftrain).get('class').dtypes

((627,), dtype('O'))

Y la serie `age`:

In [17]:
dict(dftrain).get('age').shape,dict(dftrain).get('age').dtypes

((627,), dtype('float64'))

Podemos recuperar un valor del diccionario:

In [18]:
dict(dftrain).get('class')[0],dict(dftrain).get('age')[0]

('Third', 22.0)

In [19]:
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: survived, dtype: int64

### Crear los datasets para el entrenamiento

La funcion que prepara el dataset toma como entrada el dataframe. Creara un tensor convirtiendo el dataframe en un diccionario, y luego aplicando sobre el dataset los metodos habitules para desordenar los datos, hacer lotes, transformar, ...

In [20]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Veamos la funcion

In [21]:
train_input_fn

<function __main__.make_input_fn.<locals>.input_function()>

Veamos la ejecución de la función

In [22]:
train_input_fn()

<RepeatDataset shapes: ({sex: (None,), age: (None,), n_siblings_spouses: (None,), parch: (None,), fare: (None,), class: (None,), deck: (None,), embark_town: (None,), alone: (None,)}, (None,)), types: ({sex: tf.int32, age: tf.float64, n_siblings_spouses: tf.int32, parch: tf.int32, fare: tf.float64, class: tf.string, deck: tf.string, embark_town: tf.string, alone: tf.string}, tf.int32)>

You can inspect the dataset:

In [23]:
#Obtiene un dataset
ds = make_input_fn(dftrain, y_train, batch_size=10)()

#El dataset es una dupla, con las features y el resultado esperado
#Las features son un diccionario que tiene una key para cada una de las feauture
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  #Convierte en un array de numpy
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

Some feature keys: ['sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

A batch of class: [b'Third' b'Third' b'Third' b'Third' b'First' b'Third' b'Third' b'Third'
 b'First' b'Third']

A batch of Labels: [0 0 1 0 1 1 1 1 0 0]


### Utilizar las Feature Columns

Podemos utilizar las feature columns que hemos creado antes. Lo que haremos es aplicar la feature column sobre los datos para obtener el resultado de la feature column. Por ejemplo, podemos aplicar la feature column para obtener los datos categorizados en lugar de la columna original. 

Probamos primero con la columna edad. Esta es numérica:

In [24]:
age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[  7.925 ],
       [  7.25  ],
       [ 20.575 ],
       [  0.    ],
       [227.525 ],
       [  7.2292],
       [ 12.475 ],
       [ 16.1   ],
       [ 30.    ],
       [ 46.9   ]], dtype=float32)

Hemos visto como la feature column utiliaza su nombre para identificar la columna del dataset que debe retornar. Podemos hacer lo mismo con una feature column que sea categorizada. Fijemonos como obtenermos los datos categorizados:

In [25]:
gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

After adding all the base features to the model, let's train the model. Training a model is just a single command using the `tf.estimator` API:

In [26]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

{'accuracy': 0.77272725, 'accuracy_baseline': 0.625, 'auc': 0.8362718, 'auc_precision_recall': 0.7739983, 'average_loss': 0.4742783, 'label/mean': 0.375, 'loss': 0.46532747, 'precision': 0.7532467, 'prediction/mean': 0.36155233, 'recall': 0.5858586, 'global_step': 200}


Interesante. Estamos usando un modelo previamente enlatado, en tf.estimator. El modelo lo tenemos que entrenar, pero la funcion de entrenamiento es bastante simple: tiene como argumento de entrada un método que cuando se invoca retorna un dataset.

Hay que destacar que el `LinearClassifier` tiene como argumento una colección de feature columns. Podemos ver cada una de estas feature columns como una transformación que aplicamos sobre el dataset para generar el datos que se utilice como training set.

El método que evalua el modelo también usa una funcion como entrada

In [27]:
#Tomamos un set de datos
esUnDataset =eval_input_fn().take(1)
#los incluimos en una lista
list(esUnDataset.as_numpy_iterator()) 

[({'sex': array([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
          0, 1, 0, 0, 1, 1, 1, 0, 1, 0]),
   'age': array([35., 54., 58., 55., 34., 15.,  8., 21., 18., 19., 28., 21.,  5.,
          28., 29., 16., 26., 17., 33., 29., 20., 26., 21., 38., 20.,  2.,
          21., 54., 12., 28., 33., 29.]),
   'n_siblings_spouses': array([0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 1, 0, 0, 5, 1, 0, 3, 0, 0, 1,
          0, 0, 1, 4, 2, 0, 1, 1, 0, 1]),
   'parch': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2,
          0, 0, 0, 2, 0, 1, 0, 1, 0, 0]),
   'fare': array([ 8.05  , 51.8625, 26.55  , 16.    , 13.    ,  8.0292, 21.075 ,
           8.05  , 18.    ,  7.8792,  7.75  ,  7.8   , 27.75  , 27.7208,
          10.5   , 46.9   , 14.4542, 10.5   , 15.85  ,  8.05  ,  7.8542,
          20.575 ,  7.65  ,  7.8958,  9.825 , 31.275 , 73.5   , 77.2875,
          11.2417, 22.3583,  7.8958, 26.    ]),
   'class': array([b'Third', b'First', b'First', b'Second', b

Otra forma similar de verlo:

In [28]:
esUnDataset =eval_input_fn()
list(esUnDataset.as_numpy_iterator()) 

[({'sex': array([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
          0, 1, 0, 0, 1, 1, 1, 0, 1, 0]),
   'age': array([35., 54., 58., 55., 34., 15.,  8., 21., 18., 19., 28., 21.,  5.,
          28., 29., 16., 26., 17., 33., 29., 20., 26., 21., 38., 20.,  2.,
          21., 54., 12., 28., 33., 29.]),
   'n_siblings_spouses': array([0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 1, 0, 0, 5, 1, 0, 3, 0, 0, 1,
          0, 0, 1, 4, 2, 0, 1, 1, 0, 1]),
   'parch': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2,
          0, 0, 0, 2, 0, 1, 0, 1, 0, 0]),
   'fare': array([ 8.05  , 51.8625, 26.55  , 16.    , 13.    ,  8.0292, 21.075 ,
           8.05  , 18.    ,  7.8792,  7.75  ,  7.8   , 27.75  , 27.7208,
          10.5   , 46.9   , 14.4542, 10.5   , 15.85  ,  8.05  ,  7.8542,
          20.575 ,  7.65  ,  7.8958,  9.825 , 31.275 , 73.5   , 77.2875,
          11.2417, 22.3583,  7.8958, 26.    ]),
   'class': array([b'Third', b'First', b'First', b'Second', b

In [29]:
next(esUnDataset.as_numpy_iterator())

({'sex': array([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
         0, 1, 0, 0, 1, 1, 1, 0, 1, 0]),
  'age': array([35., 54., 58., 55., 34., 15.,  8., 21., 18., 19., 28., 21.,  5.,
         28., 29., 16., 26., 17., 33., 29., 20., 26., 21., 38., 20.,  2.,
         21., 54., 12., 28., 33., 29.]),
  'n_siblings_spouses': array([0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 1, 0, 0, 5, 1, 0, 3, 0, 0, 1,
         0, 0, 1, 4, 2, 0, 1, 1, 0, 1]),
  'parch': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2,
         0, 0, 0, 2, 0, 1, 0, 1, 0, 0]),
  'fare': array([ 8.05  , 51.8625, 26.55  , 16.    , 13.    ,  8.0292, 21.075 ,
          8.05  , 18.    ,  7.8792,  7.75  ,  7.8   , 27.75  , 27.7208,
         10.5   , 46.9   , 14.4542, 10.5   , 15.85  ,  8.05  ,  7.8542,
         20.575 ,  7.65  ,  7.8958,  9.825 , 31.275 , 73.5   , 77.2875,
         11.2417, 22.3583,  7.8958, 26.    ]),
  'class': array([b'Third', b'First', b'First', b'Second', b'Second', b'Thi